# Импорт библиотек

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim import AdamW
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from sklearn.model_selection import train_test_split
from transformers.models.bert.modeling_bert import BertEncoder
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from functools import partial

# Отключаем логи WandB и Transformers
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_WANDB"] = "true"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Загрузка и предобработка данных IMDB
file_path = "/content/drive/MyDrive/IMDB Dataset.csv"
df = pd.read_csv(file_path)
df['sentiment'] = df['sentiment'].map({'negative': 0, 'positive': 1})
df = df.head(1000)
# Разбиваем данные на train, validation, test
train_val_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

#инициализация, макс длина берта - 512 токенов
class IMDBDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        review = self.data.iloc[idx]['review']
        label = self.data.iloc[idx]['sentiment']
        encoding = self.tokenizer(
            review,
            #дополняем последовательность до 512 токенов
            padding="max_length",
            #обрезаем до 512 токенов
            truncation=True,
            max_length=self.max_length,
            #возвращаем тензоры pytorch
            return_tensors="pt"
        )
        #squeeze(0) удаляет лишнюю размерность
        item = {key: tensor.squeeze(0) for key, tensor in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

# Создаем объекты датасетов и DataLoader-ов
train_dataset = IMDBDataset(train_df, tokenizer)
val_dataset = IMDBDataset(val_df, tokenizer)
test_dataset = IMDBDataset(test_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

Train size: 560
Validation size: 140
Test size: 300


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BaNAS

In [ ]:
# создание кандидатов
def synthesize_candidate_from_mask(mask: list, base_model: BertForSequenceClassification):
    candidate = copy.deepcopy(base_model)
    #все слои из оригинальной модели
    all_layers = candidate.bert.encoder.layer
    #фильтр слоев
    pruned_layers = nn.ModuleList([layer for layer, m in zip(all_layers, mask) if m == 1])
    #наличие хотя бы одного слоя
    if len(pruned_layers) == 0:
        pruned_layers = nn.ModuleList([all_layers[0]])
    # обновление
    candidate.bert.encoder.layer = pruned_layers
    candidate.config.num_hidden_layers = len(pruned_layers)
    print(f"модель со параметрами: {mask} , в ней {len(pruned_layers)} слоёв (из {len(all_layers)})")
    return candidate

In [ ]:
# маска для отбора кандидатов
def mutate_mask(mask: list, mutation_prob=0.3, min_layers=4, max_layers=12):
    new_mask = mask.copy()
    for i in range(len(new_mask)):
        if random.random() < mutation_prob:
            new_mask[i] = 1 - new_mask[i]
    #проверка ограничения по числу включённых слоёв
    num_layers = sum(new_mask)
    # мин кол-во слоев - 4
    if num_layers < min_layers:
        indices = [i for i, val in enumerate(new_mask) if val == 0]
        random.shuffle(indices)
        for i in indices[:(min_layers - num_layers)]:
            new_mask[i] = 1
    # мин кол-во слоев - 12
    elif num_layers > max_layers:
        indices = [i for i, val in enumerate(new_mask) if val == 1]
        random.shuffle(indices)
        for i in indices[:(num_layers - max_layers)]:
            new_mask[i] = 0
    return new_mask

In [ ]:
#общее количество обучаемых параметров модели
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#обучение и оценка кандидатов
def evaluate_candidate_by_mask(mask: list, base_model: BertForSequenceClassification,
                               train_loader: DataLoader, val_loader: DataLoader,
                               epochs=1, device=device,
                               min_accuracy=0.75, max_params=1e8):
    model = synthesize_candidate_from_mask(mask, base_model)
    model.to(device)
    #оптимизатор и метрика
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scaler = torch.cuda.amp.GradScaler()
    #обучение кандидатов
    model.train()
    for epoch in range(epochs):
        for batch in tqdm(train_loader, desc=f"Training candidate (mask={mask})", leave=False):
            inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(**inputs, labels=labels)
                loss = outputs.loss
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        torch.cuda.empty_cache()

    #оценка кандидатов
    model.eval()
    total, correct = 0, 0
    for batch in tqdm(val_loader, desc=f"Evaluating candidate (mask={mask})", leave=False):
        inputs = {key: value.to(device) for key, value in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)
        with torch.cuda.amp.autocast():
            outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(F.softmax(logits, dim=1), dim=1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
    #вычисление точности
    accuracy = correct / total if total > 0 else 0.0
    params_count = count_parameters(model)
    if accuracy < min_accuracy:
        print(f"Отменяем архитектуру: аккураси {accuracy:.4f} < трешхолда {min_accuracy}")
        reward = -float('inf')
    elif params_count > max_params:
        print(f"Отменяем архитектуру: параметров модели {params_count} > трешхолда {max_params}")
        reward = -float('inf')
    else:
        lambda_coef = 1e-8
        reward = accuracy - lambda_coef * params_count
    print(f"Кандидат (mask = {mask}): аккураси: {accuracy:.4f}, параметров: {params_count}, кастомная метрика: {reward:.6f}")
    return reward, accuracy, params_count

In [ ]:
#вспомогательная функция для генерации случайной маски
def create_random_mask(num_layers=12, min_layers=4, max_layers=12):
    mask = [0] * num_layers
    n = random.randint(min_layers, max_layers)
    indices = random.sample(range(num_layers), n)
    for i in indices:
        mask[i] = 1
    return mask

In [ ]:
#определение простого MLP-предсказателя (нейронного предсказателя)
class PredictorMLP(nn.Module):
    def __init__(self, input_dim=12):
        super(PredictorMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
#обучение ансамбля предсказателей на истории (history: список кортежей (mask, reward, acc, params))
def train_predictor_ensemble(history, ensemble, epochs=100, lr=1e-3):
    X = []
    y = []
    # Добавляем записи, только если конечное вознаграждение не -inf и не NaN
    for entry in history:
        mask, reward, _, _ = entry
        if np.isfinite(reward):
            X.append(mask)
            y.append(reward)
    if len(X) == 0:
        print("Нет валидных данных для обучения предсказателя.")
        return
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    criterion = nn.MSELoss()
    optimizers = [AdamW(model.parameters(), lr=lr) for model in ensemble]
    batch_size = 4
    for epoch in range(epochs):
        permutation = torch.randperm(X.size(0))
        for i in range(0, X.size(0), batch_size):
            indices = permutation[i:i+batch_size]
            batch_x = X[indices].to(device)
            batch_y = y[indices].to(device)
            for model, optimizer in zip(ensemble, optimizers):
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

In [ ]:
# предсказание значения вознаграждения для кандидата с помощью ансамбля с применением стохастического отбора (Thompson Sampling)
def predict_with_ensemble(candidate_mask, ensemble):
    candidate_tensor = torch.tensor(candidate_mask, dtype=torch.float32).unsqueeze(0).to(device)
    predictions = []
    for model in ensemble:
        with torch.no_grad():
            pred = model(candidate_tensor).item()
        predictions.append(pred)
    mean = np.mean(predictions)
    std = np.std(predictions)
    if std == 0:
        return mean
    sample = np.random.normal(mean, std)
    return sample

In [ ]:
# генерация набора кандидатов путём мутации лучших архитектур из истории
def generate_candidate_set(history, num_candidates=10, mutation_prob=0.3, min_layers=4, max_layers=12):
    sorted_history = sorted(history, key=lambda x: x[1], reverse=True)
    top_candidates = [entry[0] for entry in sorted_history[:max(1, len(sorted_history)//2)]]
    candidates = []
    while len(candidates) < num_candidates:
        parent = random.choice(top_candidates)
        child = mutate_mask(parent, mutation_prob=mutation_prob, min_layers=min_layers, max_layers=max_layers)
        candidates.append(child)
    return candidates

In [ ]:
def main_bananas():
    base_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
    base_model.to(device)

    # Гиперпараметры поиска
    num_layers = 12
    min_layers = 4
    max_layers = 12
    t0 = 5   # число начальных случайных оценок
    T = 10   # общее число оценок (итераций)
    ensemble_size = 3
    candidate_set_size = 10
    predictor_train_epochs = 50
    predictor_lr = 1e-3

    history = []

    print("Начальное случайное семплирование:")
    for i in range(t0):
        mask = create_random_mask(num_layers, min_layers, max_layers)
        reward, acc, params = evaluate_candidate_by_mask(mask, base_model, train_loader, val_loader,
                                                           epochs=1, device=device,
                                                           min_accuracy=0.55, max_params=1e8)
        history.append((mask, reward, acc, params))
        print(f"Начальный кандидат {i+1}: mask={mask}, reward={reward:.6f}, acc={acc:.4f}, params={params}")

    best_mask = None
    best_reward = -float('inf')

    #ансамбль предсказателей
    ensemble = [PredictorMLP(input_dim=num_layers).to(device) for _ in range(ensemble_size)]

    print("\nНачало итераций BANANAS:")
    for t in range(t0, T):
        print(f"\nИтерация {t+1}")
        # обучаем ансамбль на текущей истории
        train_predictor_ensemble(history, ensemble, epochs=predictor_train_epochs, lr=predictor_lr)

        # генерируем набор кандидатов
        candidates = generate_candidate_set(history, num_candidates=candidate_set_size,
                                            mutation_prob=0.3, min_layers=min_layers, max_layers=max_layers)

        # оценка кандидатов с помощью ансамбля (применяем стохастическое сэмплирование)
        candidate_scores = []
        for cand in candidates:
            score = predict_with_ensemble(cand, ensemble)
            candidate_scores.append(score)
            print(f"Candidate mask: {cand}, predicted score: {score:.6f}")

        # выбираем кандидата с максимальным предсказанным значением (так как мы максимизируем вознаграждение)
        best_idx = np.argmax(candidate_scores)
        best_candidate = candidates[best_idx]
        print(f"Выбран кандидат: {best_candidate} с predicted score {candidate_scores[best_idx]:.6f}")

        #оценка выбранного кандидата реальным обучением и валидацией
        reward, acc, params = evaluate_candidate_by_mask(best_candidate, base_model, train_loader, val_loader,
                                                           epochs=1, device=device,
                                                           min_accuracy=0.55, max_params=1e8)
        print(f"Оценка кандидата: mask={best_candidate}, reward={reward:.6f}, acc={acc:.4f}, params={params}")
        history.append((best_candidate, reward, acc, params))

        if reward > best_reward:
            best_reward = reward
            best_mask = best_candidate.copy()
            print(f"Новая лучшая архитектура: mask={best_mask} с reward={best_reward:.6f}")

    print("\n=== Поиск BANANAS завершен ===")
    print(f"Лучшая архитектура (mask): {best_mask}")
    print(f"Лучший reward: {best_reward:.6f}")

In [ ]:
if __name__ == "__main__":
    main_bananas()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Начальное случайное семплирование:
модель со параметрами: [0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1] , в ней 10 слоёв (из 12)


<ipython-input-7-24d50b49a091>:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Training candidate (mask=[0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]):   0%|          | 0/35 [00:00<?, ?it/s]

<ipython-input-7-24d50b49a091>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


...

Evaluating candidate (mask=[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1]):   0%|          | 0/9 [00:00<?, ?it/s]

Кандидат (mask = [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1]): аккураси: 0.7286, параметров: 59868674, кастомная метрика: 0.129885
Оценка кандидата: mask=[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1], reward=0.129885, acc=0.7286, params=59868674
Новая лучшая архитектура: mask=[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1] с reward=0.129885

Итерация 10
Candidate mask: [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1], predicted score: 0.002194
Candidate mask: [0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0], predicted score: -0.122320
Candidate mask: [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0], predicted score: -0.171542
Candidate mask: [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1], predicted score: 0.017137
Candidate mask: [0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1], predicted score: -0.039297
Candidate mask: [0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], predicted score: -0.170579
Candidate mask: [0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1], predicted score: -0.027768
Candidate mask: [0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1], predicted score: -0.008263
Candidate mask: [0, 0, 1,

Training candidate (mask=[0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1]):   0%|          | 0/35 [00:00<?, ?it/s]

Evaluating candidate (mask=[0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1]):   0%|          | 0/9 [00:00<?, ?it/s]

Кандидат (mask = [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1]): аккураси: 0.6429, параметров: 74044418, кастомная метрика: -0.097587
Оценка кандидата: mask=[0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1], reward=-0.097587, acc=0.6429, params=74044418

=== Поиск BANANAS завершен ===
Лучшая архитектура (mask): [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
Лучший reward: 0.129885
